In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Standard imports
import numpy as np
import xarray as xr
import tqdm as tqdm
import pandas as pd

In [3]:
# Import OpenSense modules as submodules
import sys
import os

sys.path.append(os.path.abspath("./pycomlink/"))
sys.path.append(os.path.abspath("./poligrain/src/"))
sys.path.append(os.path.abspath("./mergeplg/src/"))

import pycomlink as pycml 
import poligrain as plg
import mergeplg 

In [4]:
os.makedirs('data/adjusted_fields', exist_ok=True)

# OpenMRG adjustment

In [5]:
# OpenMRG
ds_rad = xr.open_dataset("data/andersson_2022_OpenMRG/radar/openmrg_rad.nc")                    
ds_cmls = xr.open_dataset("data/processed_cml_OpenMRG.nc").R_acc       
months = ['2015-06', '2015-07', '2015-08']                           

In [6]:
# methods parameter : default version
nnear = 12 
variogram_parameters = {"sill": 1.0, "range": 30000, "nugget": 0.3}
diff_check_sel = 5
ratio_check_sel = (0.2,8)

In [7]:
# additive IDW conservative check
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # IDW merging initialization 
    merger = mergeplg.merge.MergeDifferenceIDW(
        ds_rad=msel_ds_rad.rainfall_amount,
        ds_cmls=msel_ds_cmls,
        method="additive",
        nnear=nnear,
        range_checks={'diff_check':diff_check_sel},
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"add_p_idw": xr.concat(rainfall, dim="time") })
data.to_netcdf('data/adjusted_fields/OpenMRG_add_p_idw_cc.nc')    
del data, merger

month: 2015-06


100%|██████████| 719/719 [00:11<00:00, 61.45it/s]


month: 2015-07


100%|██████████| 744/744 [00:13<00:00, 54.24it/s]


month: 2015-08


100%|██████████| 744/744 [00:14<00:00, 52.54it/s]


In [8]:
# additive IDW no check
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # IDW merging initialization 
    merger = mergeplg.merge.MergeDifferenceIDW(
        ds_rad=msel_ds_rad.rainfall_amount,
        ds_cmls=msel_ds_cmls,
        method="additive",
        nnear=nnear,
        range_checks={},
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"add_p_idw": xr.concat(rainfall, dim="time") })
data.to_netcdf('data/adjusted_fields/OpenMRG_add_p_idw_nc.nc')    
del data, merger

month: 2015-06


100%|██████████| 719/719 [00:13<00:00, 53.28it/s]


month: 2015-07


100%|██████████| 744/744 [00:14<00:00, 52.41it/s]


month: 2015-08


100%|██████████| 744/744 [00:13<00:00, 53.94it/s]


In [9]:
# multiplicative IDW conservative check
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # IDW merging initialization 
    merger = mergeplg.merge.MergeDifferenceIDW(
        ds_rad=msel_ds_rad.rainfall_amount,
        ds_cmls=msel_ds_cmls,
        method="multiplicative",
        nnear=nnear,
        range_checks={'ratio_check':ratio_check_sel},
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"mul_p_idw": xr.concat(rainfall, dim="time") })
data.to_netcdf('data/adjusted_fields/OpenMRG_mul_p_idw_cc.nc')    
del data, merger

month: 2015-06


100%|██████████| 719/719 [00:12<00:00, 56.24it/s]


month: 2015-07


100%|██████████| 744/744 [00:13<00:00, 56.54it/s]


month: 2015-08


100%|██████████| 744/744 [00:13<00:00, 56.06it/s]


In [10]:
# multiplicative IDW no check
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # IDW merging initialization 
    merger = mergeplg.merge.MergeDifferenceIDW(
        ds_rad=msel_ds_rad.rainfall_amount,
        ds_cmls=msel_ds_cmls,
        method="multiplicative",
        nnear=nnear,
        range_checks={},
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"mul_p_idw": xr.concat(rainfall, dim="time") })
data.to_netcdf('data/adjusted_fields/OpenMRG_mul_p_idw_nc.nc')    
del data, merger

month: 2015-06


100%|██████████| 719/719 [00:13<00:00, 54.64it/s]


month: 2015-07


100%|██████████| 744/744 [00:13<00:00, 56.07it/s]


month: 2015-08


100%|██████████| 744/744 [00:13<00:00, 55.91it/s]


In [11]:
# additive POINT ORDINARY KRIGING conservative check
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # OK merging initialization
    merger = mergeplg.merge.MergeDifferenceOrdinaryKriging(
        ds_rad=msel_ds_rad.rainfall_amount,
        ds_cmls=msel_ds_cmls,
        full_line=False,
        variogram_parameters=variogram_parameters,
        nnear=nnear,
        method="additive",
        range_checks={'diff_check':diff_check_sel},
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"add_p_ok": xr.concat(rainfall, dim="time") })
data.to_netcdf('data/adjusted_fields/OpenMRG_add_p_ok_cc.nc')    
del data, merger

month: 2015-06


100%|██████████| 719/719 [00:21<00:00, 32.73it/s]


month: 2015-07


100%|██████████| 744/744 [00:26<00:00, 27.95it/s]


month: 2015-08


100%|██████████| 744/744 [00:22<00:00, 32.94it/s]


In [12]:
# additive POINT ORDINARY KRIGING no check
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # OK merging initialization
    merger = mergeplg.merge.MergeDifferenceOrdinaryKriging(
        ds_rad=msel_ds_rad.rainfall_amount,
        ds_cmls=msel_ds_cmls,
        full_line=False,
        variogram_parameters=variogram_parameters,
        nnear=nnear,
        method="additive",
        range_checks={},
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"add_p_ok": xr.concat(rainfall, dim="time") })
data.to_netcdf('data/adjusted_fields/OpenMRG_add_p_ok_nc.nc')    
del data, merger

month: 2015-06


100%|██████████| 719/719 [00:21<00:00, 32.76it/s]


month: 2015-07


100%|██████████| 744/744 [00:26<00:00, 28.16it/s]


month: 2015-08


100%|██████████| 744/744 [00:23<00:00, 31.95it/s]


In [13]:
# multiplicative POINT ORDINARY KRIGING conservative check
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # OK merging initialization
    merger = mergeplg.merge.MergeDifferenceOrdinaryKriging(
        ds_rad=msel_ds_rad.rainfall_amount,
        ds_cmls=msel_ds_cmls,
        full_line=False,
        variogram_parameters=variogram_parameters,
        nnear=nnear,
        method="multiplicative",
        range_checks={'ratio_check':ratio_check_sel},
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"mul_p_ok": xr.concat(rainfall, dim="time") })
data.to_netcdf('data/adjusted_fields/OpenMRG_mul_p_ok_cc.nc')    
del data, merger

month: 2015-06


100%|██████████| 719/719 [00:18<00:00, 38.63it/s]


month: 2015-07


100%|██████████| 744/744 [00:21<00:00, 33.82it/s]


month: 2015-08


100%|██████████| 744/744 [00:18<00:00, 40.56it/s]


In [14]:
# multiplicative POINT ORDINARY KRIGING no check
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # OK merging initialization
    merger = mergeplg.merge.MergeDifferenceOrdinaryKriging(
        ds_rad=msel_ds_rad.rainfall_amount,
        ds_cmls=msel_ds_cmls,
        full_line=False,
        variogram_parameters=variogram_parameters,
        nnear=nnear,
        method="multiplicative",
        range_checks={},
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"mul_p_ok": xr.concat(rainfall, dim="time") })
data.to_netcdf('data/adjusted_fields/OpenMRG_mul_p_ok_nc.nc')    
del data, merger

month: 2015-06


100%|██████████| 719/719 [00:21<00:00, 33.21it/s]


month: 2015-07


100%|██████████| 744/744 [00:26<00:00, 27.98it/s]


month: 2015-08


100%|██████████| 744/744 [00:22<00:00, 33.46it/s]


In [15]:
# additive BLOCK ORDINARY KRIGING conservative check
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # OK merging initialization
    merger = mergeplg.merge.MergeDifferenceOrdinaryKriging(
        ds_rad=msel_ds_rad.rainfall_amount,
        ds_cmls=msel_ds_cmls,
        full_line=True,
        variogram_parameters=variogram_parameters,
        nnear=nnear,
        method="additive",
        range_checks={'diff_check':diff_check_sel},
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"add_b_ok": xr.concat(rainfall, dim="time") })
data.to_netcdf('data/adjusted_fields/OpenMRG_add_b_ok_cc.nc')    
del data, merger

month: 2015-06


100%|██████████| 719/719 [00:21<00:00, 32.83it/s]


month: 2015-07


100%|██████████| 744/744 [00:26<00:00, 27.81it/s]


month: 2015-08


100%|██████████| 744/744 [00:19<00:00, 37.33it/s]


In [16]:
# additive BLOCK ORDINARY KRIGING no check
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # OK merging initialization
    merger = mergeplg.merge.MergeDifferenceOrdinaryKriging(
        ds_rad=msel_ds_rad.rainfall_amount,
        ds_cmls=msel_ds_cmls,
        full_line=True,
        variogram_parameters=variogram_parameters,
        nnear=nnear,
        method="additive",
        range_checks={},
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"add_b_ok": xr.concat(rainfall, dim="time") })
data.to_netcdf('data/adjusted_fields/OpenMRG_add_b_ok_nc.nc')    
del data, merger

month: 2015-06


100%|██████████| 719/719 [00:20<00:00, 35.18it/s]


month: 2015-07


100%|██████████| 744/744 [00:23<00:00, 31.03it/s]


month: 2015-08


100%|██████████| 744/744 [00:20<00:00, 35.94it/s] 


In [17]:
# multiplicative BLOCK ORDINARY KRIGING conservative check
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # OK merging initialization
    merger = mergeplg.merge.MergeDifferenceOrdinaryKriging(
        ds_rad=msel_ds_rad.rainfall_amount,
        ds_cmls=msel_ds_cmls,
        full_line=True,
        variogram_parameters=variogram_parameters,
        nnear=nnear,
        method="multiplicative",
        range_checks={'ratio_check':ratio_check_sel},
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"mul_b_ok": xr.concat(rainfall, dim="time") })
data.to_netcdf('data/adjusted_fields/OpenMRG_mul_b_ok_cc.nc')    
del data, merger

month: 2015-06


100%|██████████| 719/719 [00:18<00:00, 39.15it/s] 


month: 2015-07


100%|██████████| 744/744 [00:19<00:00, 38.70it/s] 


month: 2015-08


100%|██████████| 744/744 [00:17<00:00, 43.27it/s] 


In [18]:
# multiplicative BLOCK ORDINARY KRIGING no check
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # OK merging initialization
    merger = mergeplg.merge.MergeDifferenceOrdinaryKriging(
        ds_rad=msel_ds_rad.rainfall_amount,
        ds_cmls=msel_ds_cmls,
        full_line=True,
        variogram_parameters=variogram_parameters,
        nnear=nnear,
        method="multiplicative",
        range_checks={},
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"mul_b_ok": xr.concat(rainfall, dim="time") })
data.to_netcdf('data/adjusted_fields/OpenMRG_mul_b_ok_nc.nc')    
del data, merger

month: 2015-06


100%|██████████| 719/719 [00:21<00:00, 34.20it/s] 


month: 2015-07


100%|██████████| 744/744 [00:23<00:00, 31.57it/s] 


month: 2015-08


100%|██████████| 744/744 [00:19<00:00, 37.39it/s] 


In [19]:
# KED point conservative check
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # OK merging initialization
    merger = mergeplg.merge.MergeKrigingExternalDrift(
        ds_rad=ds_rad.rainfall_amount,
        ds_cmls=ds_cmls,
        variogram_parameters=variogram_parameters,
        nnear=nnear,
        full_line=False,
        range_checks={'diff_check':diff_check_sel},
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"ked_p": xr.concat(rainfall, dim="time") })
data.to_netcdf('data/adjusted_fields/OpenMRG_ked_p_cc.nc')    
del data, merger

month: 2015-06


100%|██████████| 719/719 [00:19<00:00, 36.50it/s] 


month: 2015-07


100%|██████████| 744/744 [00:24<00:00, 30.24it/s]


month: 2015-08


100%|██████████| 744/744 [00:23<00:00, 32.16it/s]


In [20]:
# KED point no check
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # OK merging initialization
    merger = mergeplg.merge.MergeKrigingExternalDrift(
        ds_rad=ds_rad.rainfall_amount,
        ds_cmls=ds_cmls,
        variogram_parameters=variogram_parameters,
        nnear=nnear,
        full_line=False,
        range_checks={},
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"ked_p": xr.concat(rainfall, dim="time") })
data.to_netcdf('data/adjusted_fields/OpenMRG_ked_p_nc.nc')    
del data, merger

month: 2015-06


100%|██████████| 719/719 [00:19<00:00, 36.19it/s] 


month: 2015-07


100%|██████████| 744/744 [00:25<00:00, 29.68it/s]


month: 2015-08


100%|██████████| 744/744 [00:20<00:00, 35.77it/s] 


In [21]:
# KED block conservative check
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # OK merging initialization
    merger = mergeplg.merge.MergeKrigingExternalDrift(
        ds_rad=ds_rad.rainfall_amount,
        ds_cmls=ds_cmls,
        variogram_parameters=variogram_parameters,
        nnear=nnear,
        full_line=True,
        range_checks={'diff_check':diff_check_sel},
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"ked_b": xr.concat(rainfall, dim="time") })
data.to_netcdf('data/adjusted_fields/OpenMRG_ked_b_cc.nc')    
del data, merger

month: 2015-06


100%|██████████| 719/719 [00:20<00:00, 35.80it/s] 


month: 2015-07


100%|██████████| 744/744 [00:24<00:00, 30.34it/s]


month: 2015-08


100%|██████████| 744/744 [00:19<00:00, 37.80it/s] 


In [22]:
# KED block no check
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # OK merging initialization
    merger = mergeplg.merge.MergeKrigingExternalDrift(
        ds_rad=ds_rad.rainfall_amount,
        ds_cmls=ds_cmls,
        variogram_parameters=variogram_parameters,
        nnear=nnear,
        full_line=True,
        range_checks={},
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"ked_b": xr.concat(rainfall, dim="time") })
data.to_netcdf('data/adjusted_fields/OpenMRG_ked_b_nc.nc')    
del data, merger

month: 2015-06


100%|██████████| 719/719 [00:19<00:00, 36.97it/s] 


month: 2015-07


100%|██████████| 744/744 [00:24<00:00, 30.00it/s] 


month: 2015-08


100%|██████████| 744/744 [00:20<00:00, 36.06it/s]


In [23]:
dataset = 'OpenMRG'
adj_path = 'data/adjusted_fields/'

ds_gauges = xr.open_dataset('data/andersson_2022_OpenMRG/gauges/openmrg_gauges.nc')     
ds_rad = xr.open_dataset("data/andersson_2022_OpenMRG/radar/openmrg_rad.nc")          
ds_cmls = xr.open_dataset("data/processed_cml_OpenMRG.nc").R_acc            

# adding adjusted field and naming
ds_rad = ds_rad.rename({'rainfall_amount':'radar'})  
ds_rad['add_p_idw']=xr.open_dataset(adj_path+dataset+'_add_p_idw_cc.nc').add_p_idw
ds_rad['add_p_ok']=xr.open_dataset(adj_path+dataset+'_add_p_ok_cc.nc').add_p_ok
ds_rad['add_b_ok']=xr.open_dataset(adj_path+dataset+'_add_b_ok_cc.nc').add_b_ok
ds_rad['mul_p_idw']=xr.open_dataset(adj_path+dataset+'_mul_p_idw_cc.nc').mul_p_idw
ds_rad['mul_p_ok']=xr.open_dataset(adj_path+dataset+'_mul_p_ok_cc.nc').mul_p_ok
ds_rad['mul_b_ok']=xr.open_dataset(adj_path+dataset+'_mul_b_ok_cc.nc').mul_b_ok
ds_rad['ked_p']=xr.open_dataset(adj_path+dataset+'_ked_p_cc.nc').ked_p
ds_rad['ked_b']=xr.open_dataset(adj_path+dataset+'_ked_b_cc.nc').ked_b

fields = ['radar', 'add_p_idw', 'add_p_ok', 
          'add_b_ok', 'ked_p', 'ked_b',
          'mul_p_idw', 'mul_p_ok', 'mul_b_ok']

# RAINFALL FIELDS AT THE RAIN GAUGES
get_grid_at_points = plg.spatial.GridAtPoints(
    da_gridded_data=ds_rad.isel(time = 0), 
    da_point_data=ds_gauges.isel(time = 0),
    nnear=1,
    stat="best" 
)

for field in fields:
    ds_gauges[field] = get_grid_at_points(
        da_gridded_data=ds_rad[field],
        da_point_data=ds_gauges.rainfall_amount,  
    )

# COMPUTE METRICS
threshold = 0.2

for field in fields:
        if field == 'radar':
            metric = pd.DataFrame([plg.validation.calculate_rainfall_metrics(
                reference=ds_gauges.rainfall_amount.values.flatten(),
                estimate=ds_gauges[field].values.flatten(),
                ref_thresh=threshold,
                est_thresh=threshold,
            )]) 
            metric['dataset'] = 'OpenMRG'
            metric['method'] = field
            result = metric
        else:
            metric = pd.DataFrame([plg.validation.calculate_rainfall_metrics(
                reference=ds_gauges.rainfall_amount.values.flatten(),
                estimate=ds_gauges[field].values.flatten(),
                ref_thresh=threshold,
                est_thresh=threshold,
            )]) 
            metric['dataset'] = 'OpenMRG'
            metric['method'] = field
            result = pd.concat([result, metric])

result.to_csv('metrics_OpenMRG_cc.csv')

In [24]:
dataset = 'OpenMRG'
adj_path = 'data/adjusted_fields/'

ds_gauges = xr.open_dataset('data/andersson_2022_OpenMRG/gauges/openmrg_gauges.nc')     
ds_rad = xr.open_dataset("data/andersson_2022_OpenMRG/radar/openmrg_rad.nc")          
ds_cmls = xr.open_dataset("data/processed_cml_OpenMRG.nc").R_acc        

# adding adjusted field and naming
ds_rad = ds_rad.rename({'rainfall_amount':'radar'})  
ds_rad['add_p_idw']=xr.open_dataset(adj_path+dataset+'_add_p_idw_nc.nc').add_p_idw
ds_rad['add_p_ok']=xr.open_dataset(adj_path+dataset+'_add_p_ok_nc.nc').add_p_ok
ds_rad['add_b_ok']=xr.open_dataset(adj_path+dataset+'_add_b_ok_nc.nc').add_b_ok
ds_rad['mul_p_idw']=xr.open_dataset(adj_path+dataset+'_mul_p_idw_nc.nc').mul_p_idw
ds_rad['mul_p_ok']=xr.open_dataset(adj_path+dataset+'_mul_p_ok_nc.nc').mul_p_ok
ds_rad['mul_b_ok']=xr.open_dataset(adj_path+dataset+'_mul_b_ok_nc.nc').mul_b_ok
ds_rad['ked_p']=xr.open_dataset(adj_path+dataset+'_ked_p_nc.nc').ked_p
ds_rad['ked_b']=xr.open_dataset(adj_path+dataset+'_ked_b_nc.nc').ked_b

fields = ['radar', 'add_p_idw', 'add_p_ok', 
          'add_b_ok', 'ked_p', 'ked_b',
          'mul_p_idw', 'mul_p_ok', 'mul_b_ok']

# RAINFALL FIELDS AT THE RAIN GAUGES
get_grid_at_points = plg.spatial.GridAtPoints(
    da_gridded_data=ds_rad.isel(time = 0), 
    da_point_data=ds_gauges.isel(time = 0),
    nnear=1,
    stat="best" 
)

for field in fields:
    ds_gauges[field] = get_grid_at_points(
        da_gridded_data=ds_rad[field],
        da_point_data=ds_gauges.rainfall_amount,  
    )

# COMPUTE METRICS
threshold = 0.2

for field in fields:
        if field == 'radar':
            metric = pd.DataFrame([plg.validation.calculate_rainfall_metrics(
                reference=ds_gauges.rainfall_amount.values.flatten(),
                estimate=ds_gauges[field].values.flatten(),
                ref_thresh=threshold,
                est_thresh=threshold,
            )]) 
            metric['dataset'] = 'OpenMRG'
            metric['method'] = field
            result = metric
        else:
            metric = pd.DataFrame([plg.validation.calculate_rainfall_metrics(
                reference=ds_gauges.rainfall_amount.values.flatten(),
                estimate=ds_gauges[field].values.flatten(),
                ref_thresh=threshold,
                est_thresh=threshold,
            )]) 
            metric['dataset'] = 'OpenMRG'
            metric['method'] = field
            result = pd.concat([result, metric])

result.to_csv('metrics_OpenMRG_nc.csv')

# OpenRainER adjustment

In [25]:
# OpenRainER
ds_rad = xr.open_dataset("data/covi_2024_OpenRainER/openrainer_radar.nc")         
ds_cmls = xr.open_dataset("data/processed_cml_OpenRainER.nc").R_acc      
months = ['2022-06', '2022-07', '2022-08']

In [26]:
# methods parameter : default version
nnear = 12 
variogram_parameters = {"sill": 1.0, "range": 30000, "nugget": 0.3}
diff_check_sel = 5
ratio_check_sel = (0.2,8)

In [27]:
# additive IDW conservative check
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # IDW merging initialization 
    merger = mergeplg.merge.MergeDifferenceIDW(
        ds_rad=msel_ds_rad.rainfall_amount,
        ds_cmls=msel_ds_cmls,
        method="additive",
        nnear=nnear,
        range_checks={'diff_check':diff_check_sel},
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"add_p_idw": xr.concat(rainfall, dim="time") })
data.to_netcdf('data/adjusted_fields/OpenRainER_add_p_idw_cc.nc')    
del data, merger

month: 2022-06


100%|██████████| 719/719 [00:55<00:00, 12.95it/s]


month: 2022-07


100%|██████████| 744/744 [01:01<00:00, 12.04it/s]


month: 2022-08


100%|██████████| 744/744 [01:05<00:00, 11.40it/s]


In [28]:
# additive IDW no check
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # IDW merging initialization 
    merger = mergeplg.merge.MergeDifferenceIDW(
        ds_rad=msel_ds_rad.rainfall_amount,
        ds_cmls=msel_ds_cmls,
        method="additive",
        nnear=nnear,
        range_checks={},
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"add_p_idw": xr.concat(rainfall, dim="time") })
data.to_netcdf('data/adjusted_fields/OpenRainER_add_p_idw_nc.nc')    
del data, merger

month: 2022-06


100%|██████████| 719/719 [00:53<00:00, 13.41it/s]


month: 2022-07


100%|██████████| 744/744 [00:54<00:00, 13.68it/s]


month: 2022-08


100%|██████████| 744/744 [01:02<00:00, 11.89it/s]


In [29]:
# multiplicative IDW conservative check
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # IDW merging initialization 
    merger = mergeplg.merge.MergeDifferenceIDW(
        ds_rad=msel_ds_rad.rainfall_amount,
        ds_cmls=msel_ds_cmls,
        method="multiplicative",
        nnear=nnear,
        range_checks={'ratio_check':ratio_check_sel},
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"mul_p_idw": xr.concat(rainfall, dim="time") })
data.to_netcdf('data/adjusted_fields/OpenRainER_mul_p_idw_cc.nc')    
del data, merger

month: 2022-06


100%|██████████| 719/719 [00:52<00:00, 13.68it/s]


month: 2022-07


100%|██████████| 744/744 [00:55<00:00, 13.43it/s]


month: 2022-08


100%|██████████| 744/744 [01:01<00:00, 12.08it/s]


In [30]:
# multiplicative IDW no check
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # IDW merging initialization 
    merger = mergeplg.merge.MergeDifferenceIDW(
        ds_rad=msel_ds_rad.rainfall_amount,
        ds_cmls=msel_ds_cmls,
        method="multiplicative",
        nnear=nnear,
        range_checks={},
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"mul_p_idw": xr.concat(rainfall, dim="time") })
data.to_netcdf('data/adjusted_fields/OpenRainER_mul_p_idw_nc.nc')    
del data, merger

month: 2022-06


100%|██████████| 719/719 [00:55<00:00, 12.91it/s]


month: 2022-07


100%|██████████| 744/744 [00:56<00:00, 13.18it/s]


month: 2022-08


100%|██████████| 744/744 [01:02<00:00, 11.92it/s]


In [31]:
# additive POINT ORDINARY KRIGING conservative check
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # OK merging initialization
    merger = mergeplg.merge.MergeDifferenceOrdinaryKriging(
        ds_rad=msel_ds_rad.rainfall_amount,
        ds_cmls=msel_ds_cmls,
        full_line=False,
        variogram_parameters=variogram_parameters,
        nnear=nnear,
        method="additive",
        range_checks={'diff_check':diff_check_sel},
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"add_p_ok": xr.concat(rainfall, dim="time") })
data.to_netcdf('data/adjusted_fields/OpenRainER_add_p_ok_cc.nc')    
del data, merger

month: 2022-06


100%|██████████| 719/719 [08:41<00:00,  1.38it/s]


month: 2022-07


100%|██████████| 744/744 [02:21<00:00,  5.27it/s]


month: 2022-08


100%|██████████| 744/744 [06:58<00:00,  1.78it/s]


In [32]:
# additive POINT ORDINARY KRIGING no check
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # OK merging initialization
    merger = mergeplg.merge.MergeDifferenceOrdinaryKriging(
        ds_rad=msel_ds_rad.rainfall_amount,
        ds_cmls=msel_ds_cmls,
        full_line=False,
        variogram_parameters=variogram_parameters,
        nnear=nnear,
        method="additive",
        range_checks={},
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"add_p_ok": xr.concat(rainfall, dim="time") })
data.to_netcdf('data/adjusted_fields/OpenRainER_add_p_ok_nc.nc')    
del data, merger

month: 2022-06


100%|██████████| 719/719 [08:08<00:00,  1.47it/s]


month: 2022-07


100%|██████████| 744/744 [01:58<00:00,  6.29it/s]


month: 2022-08


100%|██████████| 744/744 [05:42<00:00,  2.17it/s]


In [33]:
# multiplicative POINT ORDINARY KRIGING conservative check
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # OK merging initialization
    merger = mergeplg.merge.MergeDifferenceOrdinaryKriging(
        ds_rad=msel_ds_rad.rainfall_amount,
        ds_cmls=msel_ds_cmls,
        full_line=False,
        variogram_parameters=variogram_parameters,
        nnear=nnear,
        method="multiplicative",
        range_checks={'ratio_check':ratio_check_sel},
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"mul_p_ok": xr.concat(rainfall, dim="time") })
data.to_netcdf('data/adjusted_fields/OpenRainER_mul_p_ok_cc.nc')    
del data, merger

month: 2022-06


100%|██████████| 719/719 [03:38<00:00,  3.29it/s]


month: 2022-07


100%|██████████| 744/744 [01:02<00:00, 11.82it/s] 


month: 2022-08


100%|██████████| 744/744 [03:28<00:00,  3.57it/s]


In [34]:
# multiplicative POINT ORDINARY KRIGING no check
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # OK merging initialization
    merger = mergeplg.merge.MergeDifferenceOrdinaryKriging(
        ds_rad=msel_ds_rad.rainfall_amount,
        ds_cmls=msel_ds_cmls,
        full_line=False,
        variogram_parameters=variogram_parameters,
        nnear=nnear,
        method="multiplicative",
        range_checks={},
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"mul_p_ok": xr.concat(rainfall, dim="time") })
data.to_netcdf('data/adjusted_fields/OpenRainER_mul_p_ok_nc.nc')    
del data, merger

month: 2022-06


100%|██████████| 719/719 [08:12<00:00,  1.46it/s]


month: 2022-07


100%|██████████| 744/744 [02:34<00:00,  4.82it/s]


month: 2022-08


100%|██████████| 744/744 [07:00<00:00,  1.77it/s]


In [35]:
# additive BLOCK ORDINARY KRIGING conservative check
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # OK merging initialization
    merger = mergeplg.merge.MergeDifferenceOrdinaryKriging(
        ds_rad=msel_ds_rad.rainfall_amount,
        ds_cmls=msel_ds_cmls,
        full_line=True,
        variogram_parameters=variogram_parameters,
        nnear=nnear,
        method="additive",
        range_checks={'diff_check':diff_check_sel},
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"add_b_ok": xr.concat(rainfall, dim="time") })
data.to_netcdf('data/adjusted_fields/OpenRainER_add_b_ok_cc.nc')    
del data, merger

month: 2022-06


100%|██████████| 719/719 [08:34<00:00,  1.40it/s]


month: 2022-07


100%|██████████| 744/744 [02:04<00:00,  5.97it/s]


month: 2022-08


100%|██████████| 744/744 [06:01<00:00,  2.06it/s]


In [36]:
# additive BLOCK ORDINARY KRIGING no check
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # OK merging initialization
    merger = mergeplg.merge.MergeDifferenceOrdinaryKriging(
        ds_rad=msel_ds_rad.rainfall_amount,
        ds_cmls=msel_ds_cmls,
        full_line=True,
        variogram_parameters=variogram_parameters,
        nnear=nnear,
        method="additive",
        range_checks={},
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"add_b_ok": xr.concat(rainfall, dim="time") })
data.to_netcdf('data/adjusted_fields/OpenRainER_add_b_ok_nc.nc')    
del data, merger

month: 2022-06


100%|██████████| 719/719 [07:48<00:00,  1.54it/s]


month: 2022-07


100%|██████████| 744/744 [02:05<00:00,  5.93it/s]


month: 2022-08


100%|██████████| 744/744 [06:10<00:00,  2.01it/s]


In [37]:
# multiplicative BLOCK ORDINARY KRIGING conservative check
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # OK merging initialization
    merger = mergeplg.merge.MergeDifferenceOrdinaryKriging(
        ds_rad=msel_ds_rad.rainfall_amount,
        ds_cmls=msel_ds_cmls,
        full_line=True,
        variogram_parameters=variogram_parameters,
        nnear=nnear,
        method="multiplicative",
        range_checks={'ratio_check':ratio_check_sel},
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"mul_b_ok": xr.concat(rainfall, dim="time") })
data.to_netcdf('data/adjusted_fields/OpenRainER_mul_b_ok_cc.nc')    
del data, merger

month: 2022-06


100%|██████████| 719/719 [03:58<00:00,  3.01it/s]


month: 2022-07


100%|██████████| 744/744 [01:09<00:00, 10.75it/s] 


month: 2022-08


100%|██████████| 744/744 [03:52<00:00,  3.20it/s]


In [38]:
# multiplicative BLOCK ORDINARY KRIGING no check
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # OK merging initialization
    merger = mergeplg.merge.MergeDifferenceOrdinaryKriging(
        ds_rad=msel_ds_rad.rainfall_amount,
        ds_cmls=msel_ds_cmls,
        full_line=True,
        variogram_parameters=variogram_parameters,
        nnear=nnear,
        method="multiplicative",
        range_checks={},
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"mul_b_ok": xr.concat(rainfall, dim="time") })
data.to_netcdf('data/adjusted_fields/OpenRainER_mul_b_ok_nc.nc')    
del data, merger

month: 2022-06


100%|██████████| 719/719 [08:44<00:00,  1.37it/s]


month: 2022-07


100%|██████████| 744/744 [02:12<00:00,  5.59it/s]


month: 2022-08


100%|██████████| 744/744 [06:30<00:00,  1.90it/s]


In [39]:
# KED point conservative check
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # OK merging initialization
    merger = mergeplg.merge.MergeKrigingExternalDrift(
        ds_rad=ds_rad.rainfall_amount,
        ds_cmls=ds_cmls,
        variogram_parameters=variogram_parameters,
        nnear=nnear,
        full_line=False,
        range_checks={'diff_check':diff_check_sel},
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"ked_p": xr.concat(rainfall, dim="time") })
data.to_netcdf('data/adjusted_fields/OpenRainER_ked_p_cc.nc')    
del data, merger

month: 2022-06


100%|██████████| 719/719 [08:24<00:00,  1.43it/s]


month: 2022-07


100%|██████████| 744/744 [02:04<00:00,  6.00it/s]


month: 2022-08


100%|██████████| 744/744 [06:14<00:00,  1.98it/s]


In [40]:
# KED point no check
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # OK merging initialization
    merger = mergeplg.merge.MergeKrigingExternalDrift(
        ds_rad=ds_rad.rainfall_amount,
        ds_cmls=ds_cmls,
        variogram_parameters=variogram_parameters,
        nnear=nnear,
        full_line=False,
        range_checks={},
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"ked_p": xr.concat(rainfall, dim="time") })
data.to_netcdf('data/adjusted_fields/OpenRainER_ked_p_nc.nc')    
del data, merger

month: 2022-06


100%|██████████| 719/719 [08:16<00:00,  1.45it/s]


month: 2022-07


100%|██████████| 744/744 [01:57<00:00,  6.31it/s]


month: 2022-08


100%|██████████| 744/744 [05:51<00:00,  2.12it/s]


In [41]:
# KED block conservative check
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # OK merging initialization
    merger = mergeplg.merge.MergeKrigingExternalDrift(
        ds_rad=ds_rad.rainfall_amount,
        ds_cmls=ds_cmls,
        variogram_parameters=variogram_parameters,
        nnear=nnear,
        full_line=True,
        range_checks={'diff_check':diff_check_sel},
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"ked_b": xr.concat(rainfall, dim="time") })
data.to_netcdf('data/adjusted_fields/OpenRainER_ked_b_cc.nc')    
del data, merger

month: 2022-06


100%|██████████| 719/719 [07:40<00:00,  1.56it/s]


month: 2022-07


100%|██████████| 744/744 [01:52<00:00,  6.60it/s]


month: 2022-08


100%|██████████| 744/744 [05:40<00:00,  2.19it/s]


In [42]:
# KED block no check
rainfall = []

for month in months:
    print('month: '+month)
    msel_ds_rad = ds_rad.sel(time = month) 
    msel_ds_cmls = ds_cmls.sel(time = month) 
    # OK merging initialization
    merger = mergeplg.merge.MergeKrigingExternalDrift(
        ds_rad=ds_rad.rainfall_amount,
        ds_cmls=ds_cmls,
        variogram_parameters=variogram_parameters,
        nnear=nnear,
        full_line=True,
        range_checks={},
    )
    # merging application (loop on timesteps)
    for time in tqdm.tqdm(msel_ds_rad.time.data):
        rainfall.append(
            merger(
                da_rad=msel_ds_rad.sel(time=time).rainfall_amount,
                da_cmls=msel_ds_cmls.sel(time=time),
            )
        )
# saving the field + removing NaNs 
data = xr.Dataset({"ked_b": xr.concat(rainfall, dim="time") })
data.to_netcdf('data/adjusted_fields/OpenRainER_ked_b_nc.nc')    
del data, merger

month: 2022-06


100%|██████████| 719/719 [07:34<00:00,  1.58it/s]


month: 2022-07


100%|██████████| 744/744 [01:50<00:00,  6.73it/s]


month: 2022-08


100%|██████████| 744/744 [05:32<00:00,  2.23it/s]


In [43]:
dataset = 'OpenRainER'
adj_path = 'data/adjusted_fields/'

ds_gauges = xr.open_dataset('data/covi_2024_OpenRainER/AWS_rainfall.nc')        
ds_rad = xr.open_dataset("data/covi_2024_OpenRainER/openrainer_radar.nc")       
ds_cmls = xr.open_dataset("data/processed_cml_OpenRainER.nc").R_acc             

# adding adjusted field and naming
ds_rad = ds_rad.rename({'rainfall_amount':'radar'})  
ds_rad['add_p_idw']=xr.open_dataset(adj_path+dataset+'_add_p_idw_cc.nc').add_p_idw
ds_rad['add_p_ok']=xr.open_dataset(adj_path+dataset+'_add_p_ok_cc.nc').add_p_ok
ds_rad['add_b_ok']=xr.open_dataset(adj_path+dataset+'_add_b_ok_cc.nc').add_b_ok
ds_rad['mul_p_idw']=xr.open_dataset(adj_path+dataset+'_mul_p_idw_cc.nc').mul_p_idw
ds_rad['mul_p_ok']=xr.open_dataset(adj_path+dataset+'_mul_p_ok_cc.nc').mul_p_ok
ds_rad['mul_b_ok']=xr.open_dataset(adj_path+dataset+'_mul_b_ok_cc.nc').mul_b_ok
ds_rad['ked_p']=xr.open_dataset(adj_path+dataset+'_ked_p_cc.nc').ked_p
ds_rad['ked_b']=xr.open_dataset(adj_path+dataset+'_ked_b_cc.nc').ked_b

fields = ['radar', 'add_p_idw', 'add_p_ok', 
          'add_b_ok', 'ked_p', 'ked_b',
          'mul_p_idw', 'mul_p_ok', 'mul_b_ok']
# RAINFALL FIELDS AT THE RAIN GAUGES
get_grid_at_points = plg.spatial.GridAtPoints(
    da_gridded_data=ds_rad.isel(time = 0), 
    da_point_data=ds_gauges.isel(time = 0),
    nnear=1,
    stat="best" 
)

for field in fields:
    ds_gauges[field] = get_grid_at_points(
        da_gridded_data=ds_rad[field],
        da_point_data=ds_gauges.rainfall_amount,  
    )

# COMPUTE METRICS
threshold = 0.2

for field in fields:
        if field == 'radar':
            metric = pd.DataFrame([plg.validation.calculate_rainfall_metrics(
                reference=ds_gauges.rainfall_amount.values.flatten(),
                estimate=ds_gauges[field].values.flatten(),
                ref_thresh=threshold,
                est_thresh=threshold,
            )]) 
            metric['dataset'] = 'OpenRainER'
            metric['method'] = field
            result = metric
        else:
            metric = pd.DataFrame([plg.validation.calculate_rainfall_metrics(
                reference=ds_gauges.rainfall_amount.values.flatten(),
                estimate=ds_gauges[field].values.flatten(),
                ref_thresh=threshold,
                est_thresh=threshold,
            )]) 
            metric['dataset'] = 'OpenRainER'
            metric['method'] = field
            result = pd.concat([result, metric])

result.to_csv('metrics_OpenRainER_cc.csv')

In [44]:
dataset = 'OpenRainER'
adj_path = 'data/adjusted_fields/'

ds_gauges = xr.open_dataset('data/covi_2024_OpenRainER/AWS_rainfall.nc')        
ds_rad = xr.open_dataset("data/covi_2024_OpenRainER/openrainer_radar.nc")       
ds_cmls = xr.open_dataset("data/processed_cml_OpenRainER.nc").R_acc             

# adding adjusted field and naming
ds_rad = ds_rad.rename({'rainfall_amount':'radar'})  
ds_rad['add_p_idw']=xr.open_dataset(adj_path+dataset+'_add_p_idw_nc.nc').add_p_idw
ds_rad['add_p_ok']=xr.open_dataset(adj_path+dataset+'_add_p_ok_nc.nc').add_p_ok
ds_rad['add_b_ok']=xr.open_dataset(adj_path+dataset+'_add_b_ok_nc.nc').add_b_ok
ds_rad['mul_p_idw']=xr.open_dataset(adj_path+dataset+'_mul_p_idw_nc.nc').mul_p_idw
ds_rad['mul_p_ok']=xr.open_dataset(adj_path+dataset+'_mul_p_ok_nc.nc').mul_p_ok
ds_rad['mul_b_ok']=xr.open_dataset(adj_path+dataset+'_mul_b_ok_nc.nc').mul_b_ok
ds_rad['ked_p']=xr.open_dataset(adj_path+dataset+'_ked_p_nc.nc').ked_p
ds_rad['ked_b']=xr.open_dataset(adj_path+dataset+'_ked_b_nc.nc').ked_b

fields = ['radar', 'add_p_idw', 'add_p_ok', 
          'add_b_ok', 'ked_p', 'ked_b',
          'mul_p_idw', 'mul_p_ok', 'mul_b_ok']
# RAINFALL FIELDS AT THE RAIN GAUGES
get_grid_at_points = plg.spatial.GridAtPoints(
    da_gridded_data=ds_rad.isel(time = 0), 
    da_point_data=ds_gauges.isel(time = 0),
    nnear=1,
    stat="best" 
)

for field in fields:
    ds_gauges[field] = get_grid_at_points(
        da_gridded_data=ds_rad[field],
        da_point_data=ds_gauges.rainfall_amount,  
    )

# COMPUTE METRICS
threshold = 0.2

for field in fields:
        if field == 'radar':
            metric = pd.DataFrame([plg.validation.calculate_rainfall_metrics(
                reference=ds_gauges.rainfall_amount.values.flatten(),
                estimate=ds_gauges[field].values.flatten(),
                ref_thresh=threshold,
                est_thresh=threshold,
            )]) 
            metric['dataset'] = 'OpenRainER'
            metric['method'] = field
            result = metric
        else:
            metric = pd.DataFrame([plg.validation.calculate_rainfall_metrics(
                reference=ds_gauges.rainfall_amount.values.flatten(),
                estimate=ds_gauges[field].values.flatten(),
                ref_thresh=threshold,
                est_thresh=threshold,
            )]) 
            metric['dataset'] = 'OpenRainER'
            metric['method'] = field
            result = pd.concat([result, metric])

result.to_csv('metrics_OpenRainER_nc.csv')